In [1]:
%pip install --upgrade --quiet  langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chat_models import AzureChatOpenAI  
from langchain.chains import LLMChain
from langchain_community.utilities import SQLDatabase
from langchain.prompts.prompt import PromptTemplate
import openai_config

import warnings
warnings.filterwarnings("ignore")

In [4]:
llm = AzureChatOpenAI(api_key=openai_config.api_key, api_version=openai_config.api_version, azure_endpoint=openai_config.api_base, azure_deployment=openai_config.deployment_id)

In [5]:
_DEFAULT_TEMPLATE = """
Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [6]:
db = SQLDatabase.from_uri("sqlite:///movie.db")
chain = LLMChain(llm= llm, prompt=PROMPT)
chain.run(dict(input="Who is director of the movie with highest imdb score?", table_info=db.get_table_info(), dialect=db.dialect))

'SQLQuery: "SELECT director FROM movies ORDER BY imdb_score DESC LIMIT 1"\nSQLResult: "Sidney Lumet"\nAnswer: "The director of the movie with the highest IMDB score is Sidney Lumet."'